In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install git+git://github.com/albertbup/deep-belief-network.git
!pip install git+git://github.com/albertbup/deep-belief-network.git@master_gpu

  Cloning git://github.com/albertbup/deep-belief-network.git to /tmp/pip-req-build-4uyk3pm4
  Running command git clone -q git://github.com/albertbup/deep-belief-network.git /tmp/pip-req-build-4uyk3pm4
  Created wheel for deep-belief-network: filename=deep_belief_network-1.0.3-cp36-none-any.whl size=13464 sha256=b230253f65b6dfef6f4f465ebb6d17ccfa26ed854a7991ece8300c23d939f334
  Stored in directory: /tmp/pip-ephem-wheel-cache-rgy9tvk5/wheels/29/6d/3b/6a50cf42a32bdfaa903b17832d60d8d3e5dc4b0fd02ae8acaf
Successfully built deep-belief-network
  Cloning git://github.com/albertbup/deep-belief-network.git (to revision master_gpu) to /tmp/pip-req-build-crpsgmcs
  Running command git clone -q git://github.com/albertbup/deep-belief-network.git /tmp/pip-req-build-crpsgmcs
  Running command git checkout -b master_gpu --track origin/master_gpu
  Switched to a new branch 'master_gpu'
  Branch 'master_gpu' set up to track remote branch 'master_gpu' from 'origin'.
     |████████████████████████████████

In [ ]:
!pip install scikit-plot

In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
import scikitplot as skplt

np.random.seed(1337)  # for reproducibility
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics.classification import accuracy_score
tf.compat.v1.Session()
from dbn import SupervisedDBNClassification
#from dbn.tensorflow import SupervisedDBNClassification
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.model_selection import KFold,cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error,roc_auc_score,cohen_kappa_score)
# kfold=KFold(5,False)
# cvscores = []

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
ds= pd.read_csv('/content/drive/My Drive/code/dataset/df_spark.csv')
ds.shape

(357941, 14)

In [ ]:
ds = ds.drop(['Unnamed: 0'], axis=1)
ds.head()

# drop unnamed column

,sourceID_code,sourceAddress_code,sourceType_code,sourceLocation_code,destinationServiceAddress_code,destinationServiceType_code,destinationLocation_code,accessedNodeAddress_code,accessedNodeType_code,operation_code,value,timestamp,normality_code
0,21,30,2,3,30,2,3,59,6,2,0.0,1520031600000,7
1,32,70,2,4,68,2,4,134,6,2,0.0,1520031603269,7
2,15,1,2,2,0,2,2,0,6,2,0.0,1520031603279,7
3,33,77,2,7,73,2,7,144,6,2,0.0,1520031603290,7
4,55,78,3,7,74,3,7,146,7,2,0.0,1520031603464,7


In [ ]:
ds.shape

(357941, 13)

In [ ]:
sc = Normalizer()

y = ds.iloc[:,12].values
X = ds.iloc[:,:12].values
X = sc.fit_transform(X)

from sklearn.preprocessing import label_binarize

Y = label_binarize(y, classes=[0, 1, 2, 3, 4, 5, 6, 7])
n_classes = Y.shape[1]
print(n_classes)


from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

print(f'X_train: {X_train.shape} \n X_test: {X_test.shape} \n y_train: {y_train.shape} \n y_test: {y_test.shape}')

# split data

8
X_train: (286352, 12) 
 X_test: (71589, 12) 
 y_train: (286352,) 
 y_test: (71589,)


In [ ]:
target_names = ['DoS', 'Probing',	'MaliciousControl', 'MaliciousOperation', 'Scan', 'Spying', 'WrongSetup', 'Normal']

In [ ]:
# Training
classifier = SupervisedDBNClassification(hidden_layers_structure=[256, 256],
                                         learning_rate_rbm=0.05,
                                         learning_rate=0.1,
                                         n_epochs_rbm=10,
                                         n_iter_backprop=50,
                                         batch_size=32,
                                         activation_function='relu',
                                         dropout_p=0.2)
classifier.fit(X_train, y_train)

# Save the model
classifier.save('model.pkl')

# Restore it
classifier = SupervisedDBNClassification.load('model.pkl')


[START] Pre-training step:
>> Epoch 1 finished 	RBM Reconstruction error 0.000691
>> Epoch 2 finished 	RBM Reconstruction error 0.000350
>> Epoch 3 finished 	RBM Reconstruction error 0.000102
>> Epoch 4 finished 	RBM Reconstruction error 0.000032
>> Epoch 5 finished 	RBM Reconstruction error 0.000012
>> Epoch 6 finished 	RBM Reconstruction error 0.000012
>> Epoch 7 finished 	RBM Reconstruction error 0.000011
>> Epoch 8 finished 	RBM Reconstruction error 0.000020
>> Epoch 9 finished 	RBM Reconstruction error 0.000003
>> Epoch 10 finished 	RBM Reconstruction error 0.000006
>> Epoch 1 finished 	RBM Reconstruction error 0.001623
>> Epoch 2 finished 	RBM Reconstruction error 0.001502
>> Epoch 3 finished 	RBM Reconstruction error 0.001163
>> Epoch 4 finished 	RBM Reconstruction error 0.000476
>> Epoch 5 finished 	RBM Reconstruction error 0.000114
>> Epoch 6 finished 	RBM Reconstruction error 0.000067
>> Epoch 7 finished 	RBM Reconstruction error 0.000050
>> Epoch 8 finished 	RBM Reconstructi

In [ ]:
### train_accuracy
import sklearn.metrics as metrics
expected = y_train
predicted = classifier.predict(X_train)
data = classification_report(expected,np.round(predicted), target_names=target_names)
print(data)
conmat = confusion_matrix(expected,predicted)
print(conmat)

print("Accuracy_Train:",metrics.accuracy_score(expected, np.round(predicted)))
print("Precision_Train:",metrics.precision_score(expected, np.round(predicted), average='weighted'))
print("Recall_Train:",metrics.recall_score(expected, np.round(predicted), average='weighted'))
print("f1-score_Train:",metrics.f1_score(expected, np.round(predicted), average='weighted'))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                    precision    recall  f1-score   support

               DoS       0.00      0.00      0.00      4622
           Probing       0.00      0.00      0.00       285
  MaliciousControl       0.00      0.00      0.00       696
MaliciousOperation       0.00      0.00      0.00       641
              Scan       0.00      0.00      0.00      1215
            Spying       0.00      0.00      0.00       420
        WrongSetup       0.00      0.00      0.00        96
            Normal       0.97      1.00      0.99    278377

          accuracy                           0.97    286352
         macro avg       0.12      0.12      0.12    286352
      weighted avg       0.95      0.97      0.96    286352

[[     0      0      0      0      0      0      0   4622]
 [     0      0      0      0      0      0      0    285]
 [     0      0      0      0      0      0      0    696]
 [     0      0      0      0      0      0      0    641]
 [     0      0      0      0      0     

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
### test accuracy
import sklearn.metrics as metrics
expected = y_test
predicted = classifier.predict(X_test)
data = classification_report(expected,np.round(predicted))
print(data)
conmat = confusion_matrix(expected,predicted)
print(conmat)

print("Accuracy_Test:",metrics.accuracy_score(expected, (predicted)))
print("Precision_Test:",metrics.precision_score(expected, np.round(predicted), average='weighted'))
print("Recall_Test:",metrics.recall_score(expected, np.round(predicted), average='weighted'))
print("f1-score_Test:",metrics.f1_score(expected, np.round(predicted), average='weighted'))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1158
           1       0.00      0.00      0.00        57
           2       0.00      0.00      0.00       193
           3       0.00      0.00      0.00       164
           4       0.00      0.00      0.00       332
           5       0.00      0.00      0.00       112
           6       0.00      0.00      0.00        26
           7       0.97      1.00      0.99     69547

    accuracy                           0.97     71589
   macro avg       0.12      0.12      0.12     71589
weighted avg       0.94      0.97      0.96     71589

[[    0     0     0     0     0     0     0  1158]
 [    0     0     0     0     0     0     0    57]
 [    0     0     0     0     0     0     0   193]
 [    0     0     0     0     0     0     0   164]
 [    0     0     0     0     0     0     0   332]
 [    0     0     0     0     0     0     0   112]
 [    0     0     0     0     0     0     0